In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/workspaces/my_dissertation")


In [3]:
import pandas as pd
import geopandas as gpd
import pathlib
import numpy as np
import xarray as xr
from conceptual_runs.hydro_models import hbv, gr4j_cema_neige
from conceptual_runs.scripts.data_readers import read_gauge

In [13]:
discharge_storage = pathlib.Path("../ais_parsers/data_2/res/q_csv/").glob("*.csv")
level_storage = pathlib.Path("../ais_parsers/data_2/res/levels").glob("*.csv")

ws_file = gpd.read_file('../data/geometry/russia_ws.gpkg')
ws_file = ws_file.set_index('gauge_id')

### Extend AIS data for 2021

#### Read file and keep only calculated watershed areas

In [14]:
height_bs = pd.read_csv('../ais_parsers/data/height_id.csv',
                        index_col="Unnamed: 0")
height_bs.index.name = "gauge_id"
height_bs = height_bs[~height_bs.index.duplicated(keep='first')]
height_bs.index = height_bs.index.astype("str")

ws_file = ws_file[ws_file.index.isin(height_bs.index)]

#### Merge yearly files in single file according to gauge id

In [64]:
import pathlib
import pandas as pd
from tqdm.notebook import tqdm

for gauge_id in ws_file.index:
    try:
        old_discharge = pd.read_csv(f"../ais_parsers/data/res/ais_results/discharges/{gauge_id}.csv",
                                    index_col="date", parse_dates=True)
        old_discharge = old_discharge[~old_discharge.index.duplicated(
            keep='first')]
        old_discharge = old_discharge[['discharge']].reindex(pd.date_range(start="2008-01-01",
                                                                           end="2021-12-31"),
                                                             fill_value=np.nan)

        old_level = pd.read_csv(f"../ais_parsers/data/res/ais_results/levels/{gauge_id}.csv",
                                index_col="date", parse_dates=True).reindex(pd.date_range(start="2008-01-01",
                                                                                          end="2021-12-31"),
                                                                            fill_value=np.nan)

        new_discharge_file = pd.read_csv(f"../ais_parsers/data/discharge/final/{gauge_id}.csv",
                                         index_col="date", parse_dates=True)
        new_discharge_file = new_discharge_file[~new_discharge_file.index.duplicated(
            keep='first')].reindex(pd.date_range(start="2008-01-01",
                                                 end="2021-12-31"),
                                   fill_value=np.nan)

        new_level_file = pd.read_csv(f"../ais_parsers/data/levels/final/{gauge_id}.csv",
                                     index_col="date", parse_dates=True)

        new_level_file = new_level_file[~new_level_file.index.duplicated(
            keep='first')].reindex(pd.date_range(start="2008-01-01",
                                                 end="2021-12-31"),
                                   fill_value=np.nan)
        try:
            gauge_height = height_bs.loc[gauge_id, "height"]
        except KeyError:
            gauge_height = np.nan
        ws_area = ws_file.loc[gauge_id, "new_area"]

        old_discharge.update(new_discharge_file)
        old_level.update(new_level_file)

        old_discharge = old_discharge.rename(columns={"discharge": "q_cms_s"})
        old_discharge["q_mm_day"] = (
            old_discharge["q_cms_s"] * 86400) / (ws_area * 1e3)

        old_level = old_level.rename(columns={"level": "lvl_sm"})
        old_level["lvl_mbs"] = old_level["lvl_sm"] * 1e-2 + gauge_height

        gauge_file = pd.concat([old_discharge, old_level], axis=1)
        gauge_file.index.name = "date"
        gauge_file.to_csv(f"../ais_parsers/results_2024/{gauge_id}.csv")

    except FileNotFoundError:
        continue

### Check HBV, GR4J results and made comparison with LSTM

In [221]:
calibration_place = pathlib.Path("../conceptual_runs/cal_res")
hbv_calibration = pathlib.Path(f"{calibration_place}/hbv_full")
hbv_calibration.mkdir(exist_ok=True, parents=True)
gr4j_calibration = pathlib.Path(f"{calibration_place}/gr4j_full")
gr4j_calibration.mkdir(exist_ok=True, parents=True)

In [226]:
def read_gauge(gauge_id: str,
               simple: bool = False):

    test_df = xr.open_dataset(
        f'../data/ws_related_meteo/nc_all_q/{gauge_id}.nc').to_dataframe(
    )[['q_mm_day', 'prcp_e5l', 't_min_e5l', 't_max_e5l', 'Ep']]

    if simple:
        test_df.index.name = 'Date'

        train = test_df[:'2018']
        test = test_df['2018':]
    else:
        test_df['Temp'] = test_df[['t_max_e5l', 't_min_e5l']].mean(axis=1)
        test_df = test_df.rename(columns={'prcp_e5l': 'Prec',
                                          'Ep': 'Evap',
                                          'q_mm_day': 'Q_mm'})
        test_df.index.name = 'Date'
        test_df = test_df.drop(['t_min_e5l', 't_max_e5l'], axis=1)
        test_df.loc[test_df['Evap'] < 0, 'Evap'] = 0

        train = test_df[:'2018']
        test = test_df['2018':]

    return train, test


for gauge_id in ws_file.index:
    try:
        train, test = read_gauge(gauge_id=gauge_id,
                                 simple=False)
        lines_gr4j = open(f"{gr4j_calibration}/{gauge_id}",
                          "r").read().splitlines()
        try:
            params_gr4j = eval(lines_gr4j[3].split(":")[1])[0]
        except SyntaxError:
            params_gr4j = eval(lines_gr4j[3].split(":")[1] + "]")[0]

        lines_hbv = open(f"{hbv_calibration}/{gauge_id}",
                         "r").read().splitlines()
        params_hbv = eval(lines_hbv[3].split(":")[1])[0]

        test["Q_gr4j"] = gr4j_cema_neige.simulation(
            data=test, params=params_gr4j)
        test["Q_hbv"] = hbv.simulation(data=test, params=params_hbv)

        test = test[["Q_mm", "Q_gr4j", "Q_hbv"]]

        test.to_csv(f"../conclusions/results/hbv_gr4j_2024/{gauge_id}.csv")

    except FileNotFoundError:
        continue

### Plot HBV, GR4J results vs LSTM and LSTM-static
#### Select top-100 gauges for GR4J and HBV

In [5]:
from visualizations.scripts.drawer import metric_viewer
from visualizations.scripts.drawer import russia_plots, russia_plots_n
from conceptual_runs.scripts.data_readers import metric_df

In [15]:
rfr_cv_table = pd.read_csv("./results/res_rfr_cv.csv",
                           dtype={"gauge_id": str},
                           index_col="gauge_id")

In [16]:
rfr_table = list()
for gauge_id in ws_file.index:
    try:
        pred_df = pd.read_csv(f"./results/rfr/{gauge_id}.csv",
                              index_col="date", parse_dates=True)
        obs, pred = pred_df["q_mm_day"], pred_df["q_mm_rfr"]
        gauge_metric = metric_df(gauge_id=gauge_id,
                                 predictions=pred, targets=obs)
        rfr_table.append(gauge_metric)
    except FileNotFoundError:
        continue
rfr_table = pd.concat(rfr_table)

In [22]:
hbv_table = list()
gr4j_table = list()
for gauge_id in ws_file.index:
    try:
        pred_df = pd.read_csv(f"./results/hbv_gr4j_2024/{gauge_id}.csv",
                              index_col="Date", parse_dates=True)
        obs, pred_hbv, pred_gr4j = pred_df["Q_mm"], pred_df["Q_hbv"], pred_df["Q_gr4j"]
        gr4j_metric = metric_df(gauge_id=gauge_id,
                                predictions=pred_gr4j, targets=obs)
        gr4j_table.append(gr4j_metric)
        hbv_metric = metric_df(gauge_id=gauge_id,
                               predictions=pred_hbv, targets=obs)
        hbv_table.append(hbv_metric)
    except FileNotFoundError:
        continue
gr4j_table = pd.concat(gr4j_table)
hbv_table = pd.concat(hbv_table)

In [35]:
gr4j_ayzel = pd.read_csv("./tables/GR4J_ayzel_res.csv",
                         dtype={"gauge_id": str},
                         index_col="gauge_id")
hbv_ayzel = pd.read_csv("./tables/HBV_ayzel_res.csv",
                                 dtype={"gauge_id": str},
                                 index_col="gauge_id")

In [55]:
hbv_comparison = pd.DataFrame()
hbv_comparison["NSE_ayzel"] = hbv_ayzel[hbv_ayzel.index.isin(
    hbv_table.index)]["NSE"]
hbv_comparison["NSE_abramov"] = hbv_table[hbv_table.index.isin(
    hbv_ayzel.index)]["NSE"]

gr4j_comparison = pd.DataFrame()
gr4j_comparison["NSE_ayzel"] = gr4j_ayzel[gr4j_ayzel.index.isin(
    gr4j_table.index)]["NSE"]
gr4j_comparison["NSE_abramov"] = gr4j_table[gr4j_table.index.isin(
    gr4j_ayzel.index)]["NSE"]

In [74]:
lstm_table = pd.read_csv("./tables/LSTM_static_res.csv",
                         dtype={"gauge_id": str},
                         index_col="gauge_id")
lstm_no_static = pd.read_csv("./tables/LSTM_no_static_res.csv",
                             dtype={"gauge_id": str},
                             index_col="gauge_id")

In [93]:
align_index = lstm_table.index

hbv_table = hbv_table.loc[hbv_table.index.isin(align_index), :]
gr4j_table = gr4j_table.loc[gr4j_table.index.isin(align_index), :]
lstm_no_static = lstm_no_static.loc[lstm_no_static.index.isin(align_index), :]
rfr_table = rfr_table.loc[rfr_table.index.isin(align_index), :]
rfr_cv_table = rfr_cv_table.loc[rfr_cv_table.index.isin(align_index), :]

In [88]:
gauges_file = gpd.read_file("../data/geometry/russia_gauges.gpkg",
                            index_col="gauge_id", dtype={"gauge_id": str})
gauges_file = gauges_file.set_index("gauge_id")
gauges_file = gauges_file[gauges_file.index.isin(align_index)]

basemap_data = gpd.read_file(
    '../data/geometry/basemap_2023.gpkg')

basin_districts = gpd.read_file(
    '../data/geometry/basin_districts.gpkg')
basin_districts = basin_districts.reset_index(drop=True)
# fixed umgs shapes
basin_districts = basin_districts.reindex([0, 1, 2, 3, 4, 5, 6, 7, 8, 26, 10,
                                           11, 12, 27, 14, 15, 16, 17, 18, 19,
                                           20, 21, 22, 23, 24, 25, 9, 13])
basin_districts = basin_districts.reset_index(drop=True)
basin_districts = basin_districts.rename(columns={'BAS_OKRUG': 'name1'})

In [119]:
nse_file = gauges_file[["geometry"]]
nse_file.loc[:, ["NSE_gr4j", "NSE_hbv", "NSE_lstm",
                 "NSE_lstm_ns", "NSE_rfr", "NSE_rfr_cv"]] = pd.concat(
              [gr4j_table["NSE"], hbv_table["NSE"], lstm_table["NSE"],
               lstm_no_static["NSE"], rfr_table["NSE"], rfr_cv_table["NSE"]],
              axis=1).values
nse_file = nse_file.dropna()
# nse_file = nse_file.sort_values(by="NSE_hbv", ascending=False)[:150]
# give ugms some shine
for i, geom in enumerate(basin_districts['geometry']):

    ugms_part = nse_file.loc[[geom.intersects(gage_point)
                              for gage_point in nse_file['geometry']], :]
    ugms_name = basin_districts.loc[i, 'name1']

    basin_districts.loc[i, ['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                            'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']] = ugms_part[['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                                                                                  'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']].median()
basin_districts.loc[[25, 26, 27], ['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                                   'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']] = np.nan

# give ugms some shine
for i, geom in enumerate(basin_districts['geometry']):

    ugms_part = nse_file.loc[[geom.intersects(gage_point)
                              for gage_point in nse_file['geometry']], :]
    ugms_name = basin_districts.loc[i, 'name1']

    basin_districts.loc[i, ['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                            'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']] = ugms_part[['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                                                                                  'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']].median()
basin_districts.loc[[25, 26, 27], ['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                                   'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv']] = np.nan

In [122]:
meteo_nse = russia_plots_n(gdf_to_plot=nse_file,
                           basemap_data=basemap_data,
                           figsize=(15, 8),
                           nrows=2,
                           ncols=3,
                           label_list=['а)', 'б)', 'в)', 'г)', 'д)', 'е)'],
                           columns_from_gdf=['NSE_gr4j', 'NSE_hbv', 'NSE_lstm',
                                             'NSE_lstm_ns', 'NSE_rfr', 'NSE_rfr_cv'],
                           with_histogram=True,
                           title_text=[f'NSE для HBV,\nNSE = {nse_gr4j:.2f}',
                                       f'NSE для GR4J,\nNSE = {nse_hbv:.2f}',
                                       f'NSE для LSTM с осадками ERA5-Land,\nNSE = {nse_lstm:.2f}',
                                       f'NSE для LSTM с осадками ERA5-Land без физ.гео параметров,\nNSE = {nse_lstm_ns:.2f}',
                                       f'NSE для Random Forest Regressor,\nNSE = {nse_rfr:.2f}',
                                       f'NSE для Random Forest Regressor (cross-validation),\nNSE = {nse_rfr_cv:.2f}'],
                           ugms=True, ugms_gdf=basin_districts)